In [86]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 16 21:27:50 2021

@author: xiaoxu
"""


import sympy as smp
from sympy import Number, Add, Mul, Pow, Basic, latex, Symbol, I
from sympy.physics.secondquant import  Dagger
from copy import deepcopy
from printable import Basic as Printable,  B, Bd
from terms import Term, Terms, Annilator, Creator, Exp, t
import pickle

class Kamiltonian(Terms):

    Ks = {}

    @classmethod
    def get(cls, n, k):
        """
        get Kamiltonian K(n)_[k] according to the recursive formula.
        """
        key = str([n,k])
        # print(key)

        if key in cls.Ks.keys():
            return cls.Ks[key]

        if n == 0 and k == 0:#base case
            raise Exception("Base case of recursion not specified")

        elif n == 0 and k == 1:
            cls.Ks[key] = S(1).dot().simplify()

        elif n != 0 and k == 1:
            term1 = S(n+1).dot()
            term2 = S(n).lie_der(K(0,0))

            cls.Ks[key] = (term1 + term2).simplify()
            #this can be alternatively expressed as static of term2 - rotating
            #parts of other K(n,k'!=k)

        elif k > 1 and k <= n+1:
            # print("general")
            terms = Terms()
            for m in range(n):
                # print(m,k-1)
                terms += S(n-m).lie_der(K(m,k-1))/smp.S(k)
            cls.Ks[key] = terms.simplify()
        else:
            # print("zero case")
            return Terms()
        # cls.Ks[key].simplify()
        return cls.Ks[key]

    @classmethod
    def set_H(cls, H):
        cls.Ks[str([0,0])] = H.simplify()

    @classmethod
    def dump(cls, fdir):
        with open(fdir,'wb') as f:
            pickle.dump(cls.Ks,f)

    @classmethod
    def load(cls, fdir):
        with open(fdir,'rb') as f:
            cls.Ks = pickle.load(f)


class Generator(Terms):
    Ss = {}

    @classmethod
    def get(cls, n):
        """
        get generator S(n) according to the recursive formula.
        """
        if n in cls.Ss.keys(): return cls.Ss[n]

        # print("S"+str(n))
        terms = Terms()
        if n == 0: return Terms()
        if n == 1:
            terms += smp.S(-1)*K(0,0)
        if n > 1:
            terms += smp.S(-1)*S(n-1).lie_der(K(0,0))
        for k in range(2, n+1):
            # print(k, n-1)
            terms += smp.S(-1)*K(n-1,k)
        terms = terms.rot().integrate()

        cls.Ss[n] = terms.simplify()
        # print("S"+str(n)+"done")
        return cls.Ss[n]

    @classmethod
    def dump(cls, fdir):
        with open(fdir,'wb') as f:
            pickle.dump(cls.Ss,f)

    @classmethod
    def load(cls, fdir):
        with open(fdir,'rb') as f:
            cls.Ss = pickle.load(f)


def K(n,k = -1):
    if k!= -1: return Kamiltonian.get(n,k)
    Kn = Terms()
    for ki in range(n+2):
        Kn += Kamiltonian.get(n,ki)
    return Kn

def S(n):
    return Generator.get(n)

# Term.set_lambda(smp.S(1))

hbar = Symbol("hbar", real=True)

omega0 = Symbol('\omega_a')
omega1 = Symbol('\omega_b')
g3 = Symbol('g_3')
g4 = Symbol('g_4')
g5 = Symbol('g_5')
g6 = Symbol('g_6')
g7 = Symbol('g_7')
g8 = Symbol('g_8')
delta = Symbol('\delta')



b = Annilator('a')*Annilator('b')
br = b*Exp(-1*I*omega1*t)
bd = Creator('a')
bdr = bd*Exp(1*I*omega1*t)

a = Annilator('a')
ar = a*Exp(-1*I*omega0*t)
ad = Creator('a')
adr = ad*Exp(1*I*omega0*t)

pi = Symbol('\Pi ', complex=True)
pir = pi*Exp(-1*I*(omega0-omega1)*t)
pis = pi.conjugate()
pisr = pis*Exp(1*I*(omega0-omega1)*t)
H =  g4*((ar+adr)**4) + g3*((ar+adr)**3) +g5*(ar+adr)**5+g6*(ar+adr)**6+ g7*(ar+adr)**7+g8*(ar+adr)**8
#H = g3*(ar+adr)**3+ g4*(ar+adr)**4

Kamiltonian.set_H(H)




Exception: expression to be normal ordered contains multiple                             kinds of bosons. Not Implemented

In [85]:
S(1)

Terms(Term(Prefactor((-I*g_4/4 - 15*I*g_6*hbar/4 - 105*I*g_8*hbar**2/2, \omega_a)), B(a)**4, exp(-4*I*\omega_a*t)), Term(Prefactor((-2*I*g_4 - 30*I*g_6*hbar - 420*I*g_8*hbar**2, \omega_a)), Bd(a)*B(a)**3, exp(-2*I*\omega_a*t)), Term(Prefactor((-3*I*g_4*hbar - 45*I*g_6*hbar**2/2 - 210*I*g_8*hbar**3, \omega_a)), B(a)**2, exp(-2*I*\omega_a*t)), Term(Prefactor((3*I*g_4*hbar + 45*I*g_6*hbar**2/2 + 210*I*g_8*hbar**3, \omega_a)), Bd(a)**2, exp(2*I*\omega_a*t)), Term(Prefactor((2*I*g_4 + 30*I*g_6*hbar + 420*I*g_8*hbar**2, \omega_a)), Bd(a)**3*B(a), exp(2*I*\omega_a*t)), Term(Prefactor((I*g_4/4 + 15*I*g_6*hbar/4 + 105*I*g_8*hbar**2/2, \omega_a)), Bd(a)**4, exp(4*I*\omega_a*t)), Term(Prefactor((-I*g_3/3 - 10*I*g_5*hbar/3 - 35*I*g_7*hbar**2, \omega_a)), B(a)**3, exp(-3*I*\omega_a*t)), Term(Prefactor((-3*I*g_3 - 30*I*g_5*hbar - 315*I*g_7*hbar**2, \omega_a)), Bd(a)*B(a)**2, exp(-I*\omega_a*t)), Term(Prefactor((-3*I*g_3*hbar - 15*I*g_5*hbar**2 - 105*I*g_7*hbar**3, \omega_a)), B(a), exp(-I*\omega_a*t

In [76]:
(K(1)+K(0)).simplify()

Terms(Term(Prefactor((-11*g_3**2*hbar**2 - 130*g_3*g_5*hbar**3 - 1050*g_3*g_7*hbar**4 - 42*g_4**2*hbar**3 - 720*g_4*g_6*hbar**4 - 7560*g_4*g_8*hbar**5 - 449*g_5**2*hbar**4 - 8358*g_5*g_7*hbar**5 - 3495*g_6**2*hbar**5 - 82320*g_6*g_8*hbar**6 - 44379*g_7**2*hbar**6 - 540120*g_8**2*hbar**7, \omega_a), (3*g_4*hbar**2 + 15*g_6*hbar**3 + 105*g_8*hbar**4, 1)), 1, 1), Term(Prefactor((-1260*g_3*g_7 - 660*g_4*g_6 - 37800*g_4*g_8*hbar - 630*g_5**2 - 69300*g_5*g_7*hbar - 19650*g_6**2*hbar - 1540000*g_6*g_8*hbar**2 - 1268190*g_7**2*hbar**2 - 26219900*g_8**2*hbar**3, \omega_a), (70*g_8, 1)), Bd(a)**4*B(a)**4, 1), Term(Prefactor((-280*g_3*g_5 - 10080*g_3*g_7*hbar - 68*g_4**2 - 5280*g_4*g_6*hbar - 146160*g_4*g_8*hbar**2 - 5040*g_5**2*hbar - 257040*g_5*g_7*hbar**2 - 75100*g_6**2*hbar**2 - 3785600*g_6*g_8*hbar**3 - 2938320*g_7**2*hbar**3 - 46589200*g_8**2*hbar**4, \omega_a), (20*g_6 + 560*g_8*hbar, 1)), Bd(a)**3*B(a)**3, 1), Term(Prefactor((-30*g_3**2 - 1260*g_3*g_5*hbar - 21000*g_3*g_7*hbar**2 - 306*g_

In [77]:
(K(1)+K(0)).select(a**3).simplify()

Terms()

In [78]:
(K(1)+K(0)).select(ad**2*a**2).simplify()

Terms(Term(Prefactor((-30*g_3**2 - 1260*g_3*g_5*hbar - 21000*g_3*g_7*hbar**2 - 306*g_4**2*hbar - 11340*g_4*g_6*hbar**2 - 204120*g_4*g_8*hbar**3 - 10220*g_5**2*hbar**2 - 325080*g_5*g_7*hbar**3 - 102150*g_6**2*hbar**3 - 3742200*g_6*g_8*hbar**4 - 2629746*g_7**2*hbar**4 - 35603400*g_8**2*hbar**5, \omega_a), (6*g_4 + 90*g_6*hbar + 1260*g_8*hbar**2, 1)), Bd(a)**2*B(a)**2, 1))

In [41]:
(K(0)+K(1)+K(2)).simplify().select(ad**2*a**2)

Terms(Term(Prefactor((6*g_4 + 90*g_6*hbar + 1260*g_8*hbar**2, 1), (-30*g_3**2 - 1260*g_3*g_5*hbar - 21000*g_3*g_7*hbar**2 - 306*g_4**2*hbar - 11340*g_4*g_6*hbar**2 - 204120*g_4*g_8*hbar**3 - 10220*g_5**2*hbar**2 - 325080*g_5*g_7*hbar**3 - 102150*g_6**2*hbar**3 - 3742200*g_6*g_8*hbar**4 - 2629746*g_7**2*hbar**4 - 35603400*g_8**2*hbar**5, \omega_a), (8100*g_3**2*g_4*hbar + 174540*g_3**2*g_6*hbar**2 + 3459960*g_3**2*g_8*hbar**3 + 309480*g_3*g_4*g_5*hbar**2 + 5700240*g_3*g_4*g_7*hbar**3 + 6153480*g_3*g_5*g_6*hbar**3 + 120869840*g_3*g_5*g_8*hbar**4 + 112543872*g_3*g_6*g_7*hbar**4 + 2265127200*g_3*g_7*g_8*hbar**5 + 25164*g_4**3*hbar**2 + 1564560*g_4**2*g_6*hbar**3 + 32409720*g_4**2*g_8*hbar**4 + 2714040*g_4*g_5**2*hbar**3 + 98630784*g_4*g_5*g_7*hbar**4 + 32148900*g_4*g_6**2*hbar**4 + 1354963680*g_4*g_6*g_8*hbar**5 + 915279372*g_4*g_7**2*hbar**5 + 14747216400*g_4*g_8**2*hbar**6 + 52888720*g_5**2*g_6*hbar**4 + 1054332720*g_5**2*g_8*hbar**5 + 1955915640*g_5*g_6*g_7*hbar**5 + 40426091040*g_5*g_7

In [4]:
K(1)

Terms()